# Chapter 9: Running TensorFlow

This chapter introduces how to run TensorFlow on your computer, and how to create a computing graph with TensorFlow. For further details on the APIs and higher level usages, please refer to official documentation or other TensorFlow tutorials.

> This jupyter notebook contains my solution to coding exercises of this chapter. For answers to the questions, please check the markdown file under the same folder.

## Exercise 9: TensorFlow on Moon dataset

Requirement: Use minibatch gradient descent to realize logistic regression. Train and evaluate on the Moon Dataset.

First of all, let's do some preparation before completing the tasks below.

In [3]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import numpy as np

data, target = make_moons(n_samples=10000, noise=0.4)
X_train, X_test, y_train, y_test = train_test_split(data, target)
y_train = np.reshape(y_train, (-1, 1))
y_test = np.reshape(y_test, (-1, 1))

# set up hyperparameters
batch_size = 64
num_epochs = 3000
learning_rate = 0.001

num_train_samples, num_features = X_train.shape[0], X_train.shape[1]
num_batches = int(np.ceil(num_train_samples / batch_size))

current_epoch = -1
random_indexes = np.arange(num_train_samples)
np.random.shuffle(random_indexes)

**Task 1: Define a reusable graph in method ```logistic_regression()```.**

In [4]:
import tensorflow as tf

def logistic_regression(X):
    with tf.name_scope("logistic_regression") as scope:
        W = tf.Variable(tf.random_uniform([2, 1], -1.0, 1.0), name="weights")
        b = tf.Variable(0.0, name="bias")
        output_tensor = tf.sigmoid(tf.add(tf.matmul(X, W), b))
        
    return output_tensor

**Task 2: During the training progress, use ```Saver``` to save the checkpoint on time, and save the final model.**

In [ ]:
import os
from sklearn.metrics import accuracy_score, roc_auc_score

input_tensor = tf.placeholder(tf.float32, shape=(None, 2), name="input_tensor")
label = tf.placeholder(tf.float32, shape=(None, 1), name="label")
y = logistic_regression(input_tensor)

with tf.name_scope('loss') as scope:
    error = y - label
    mse = tf.reduce_mean(tf.square(error), name="mse")
    
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

mse_summary = tf.summary.scaler('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

with tf.Session() as sess:
    sess.run(init)
    
    # check whether there is an existing model
    if os.path.exists("/Users/Antinomy/Desktop/tmp/checkpoint"):
        saver.restore(sess, "/Users/Antinomy/Desktop/tmp/my_model.ckpt")
    
    for i in range(num_epochs):
        for j in range(num_batches):
            if i != current_epoch:
                random_indexes = np.arange(num_train_samples)
                np.random.shuffle(random_indexes)
                current_epoch += 1
                
            selected_indexes = random_indexes[(j*batch_size):((j+1)*batch_size)]
            X_train_batch, y_train_batch = X_train[selected_indexes], y_train[selected_indexes]
            if j % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={input_tensor: X_train_batch, label: y_train_batch})
                step = i*num_batches + j
                file_writer.add_summary(summary_str, step)
            
            sess.run(training_op, feed_dict={input_tensor: X_train_batch, label: y_train_batch})
        
        print("---------- Epoch %d ----------" % i)
        print("Loss:", mse.eval(feed_dict={input_tensor: X_train_batch, label: y_train_batch}))
        save_path = saver.save(sess, "/Users/Antinomy/Desktop/tmp/my_model.ckpt")
        
    saver.save(sess, "/Users/Antinomy/Desktop/final/my_final_model.ckpt")
    file_writer.close()
    
    # make predictions with the trained graph
    predictions = y.eval(feed_dict={input_tensor: X_test})
    final_predictions = [int(item > 0.5) for item in predictions]
    print("========== Result on Test Set ==========")
    print("Accuracy:", accuracy_score(y_test, final_predictions))
    print("AUC:", roc_auc_score(y_test, predictions))

**Task 3: If the training progress has been terminated, restore the saved model.**

The code is shown below.
``` python
if os.path.exists("/Users/Antinomy/Desktop/tmp/checkpoint"):
        saver.restore(sess, "/Users/Antinomy/Desktop/tmp/my_model.ckpt")
```
The code snippet first checks whether the checkpoint file exists, then use ```tf.train.Saver``` to restore the previous session.

**Task 4: Define the graph with appropriate domain to make it more decent in TensorBoard.**

Let's first check how the graph looks like in TensorBoard.

In [ ]:
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "/Users/Antinomy/Desktop/tf_logs"
logdir = "{}/run-{}".format(root_logdir, now)

To make the graph more decent, we can organize different operators under the same namespace. The code is shown below.
``` python
[...]
def logistic_regression(X):
    with tf.name_scope("logistic_regression") as scope:
        W = tf.Variable(tf.random_uniform([2, 1], -1.0, 1.0), name="weights")
        b = tf.Variable(0.0, name="bias")
        output_tensor = tf.sigmoid(tf.add(tf.matmul(X, W), b))
        
    return output_tensor

[...]
with tf.name_scope('loss') as scope:
    error = y - label
    mse = tf.reduce_mean(tf.square(error), name="mse")
[...]
```

*The complete code is shown below.* Hyperparameters are not perfectly tuned but the accuracy score and AUC score are both acceptable.

In [ ]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np
import tensorflow as tf
import os
from datetime import datetime

# prepare for dataset
data, target = make_moons(n_samples=10000, noise=0.4)
X_train, X_test, y_train, y_test = train_test_split(data, target)
y_train = np.reshape(y_train, (-1, 1))
y_test = np.reshape(y_test, (-1, 1))

# set up hyperparameters
batch_size = 64
num_epochs = 3000
learning_rate = 0.001

num_train_samples, num_features = X_train.shape[0], X_train.shape[1]
num_batches = int(np.ceil(num_train_samples / batch_size))

current_epoch = -1
random_indexes = np.arange(num_train_samples)
np.random.shuffle(random_indexes)

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "/Users/Antinomy/Desktop/tf_logs"
logdir = "{}/run-{}".format(root_logdir, now)

# input_tensor = tf.placeholder(tf.float32, shape=(None, num_features), name="input_tensor")
# label = tf.placeholder(tf.float32, shape=(None, 1), name="label")

# define computation graph
def logistic_regression(X):
    W = tf.Variable(tf.random_uniform([2, 1], -1.0, 1.0), name="weights")
    b = tf.Variable(0.0, name="bias")
    output_tensor = tf.sigmoid(tf.add(tf.matmul(X, W), b))
    return output_tensor

input_tensor = tf.placeholder(tf.float32, shape=(None, 2), name="input_tensor")
label = tf.placeholder(tf.float32, shape=(None, 1), name="label")
y = logistic_regression(input_tensor)
error = y - label
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

with tf.Session() as sess:
    sess.run(init)
    
    if os.path.exists("/Users/Antinomy/Desktop/tmp/checkpoint"):
        saver.restore(sess, "/Users/Antinomy/Desktop/tmp/my_model.ckpt")
    
    for i in range(num_epochs):
        for j in range(num_batches):
            if i != current_epoch:
                random_indexes = np.arange(num_train_samples)
                np.random.shuffle(random_indexes)
                current_epoch += 1
                
            selected_indexes = random_indexes[(j*batch_size):((j+1)*batch_size)]
            X_train_batch, y_train_batch = X_train[selected_indexes], y_train[selected_indexes]
            if j % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={input_tensor: X_train_batch, label: y_train_batch})
                step = i*num_batches + j
                file_writer.add_summary(summary_str, step)
            
            sess.run(training_op, feed_dict={input_tensor: X_train_batch, label: y_train_batch})
        
        print("---------- Epoch %d ----------" % i)
        print("Loss:", mse.eval(feed_dict={input_tensor: X_train_batch, label: y_train_batch}))
        save_path = saver.save(sess, "/Users/Antinomy/Desktop/tmp/my_model.ckpt")
        
    saver.save(sess, "/Users/Antinomy/Desktop/final/my_final_model.ckpt")
    file_writer.close()
    
    # make predictions with the trained graph
    predictions = y.eval(feed_dict={input_tensor: X_test})
    final_predictions = [int(item > 0.5) for item in predictions]
    print("========== Result on Test Set ==========")
    print("Accuracy:", accuracy_score(y_test, final_predictions))
    print("AUC:", roc_auc_score(y_test, predictions))